
## 朴素贝叶斯（Navie Bayesian Model）
前提性假设：独立性假设

隐患：训练集数据不足导致概率为0，不够合理。

修改：在分子分母加上一个常数——拉普拉斯平滑（Laplace Smoothing）

# 文本分类

## 贝叶斯文本分类
在文本分类中，将一句话一条短信中出现的词语作为该文本的特征。

将相似的特征放在一个列表中，当一句话中出现列表中的词语时，计算是广告还是正常信息的概率。

1. 将句子拆解成一个个的词——分词操作，得到单独词语和出现的概率

#### 词袋模型：只是把文本切成若干个词，不考虑词的顺序和语法
sklearn.feature_extraction.text提供了文本向量化的工具

2. 向量化
将每个句子的特征词放入列表作为特征

3. 使用朴素贝叶斯分类器，对文本进行分类
sklearn.naive_bayes.MultinominalNB, 专门针对离散型数据集的多项式分布朴素贝叶斯。

**首先用已标记好的数据集对模型进行训练**
P（是广告｜包含发票与折扣）=P（广告的概率）X P（广告中发票概率）X P（广告中折扣概率）/ P（发票，折扣）

In [9]:
import pandas as pd
# 设定导入的列，第一行作为列名，第一列作为索引
df1 = pd.read_excel('trec06cn-1.xlsx', usecols=[0,2,3,4],header=[0],index_col=[0])
df2 = pd.read_excel('trec06cn-2.xlsx', usecols=[0,2,3,4],header=[0],index_col=[0])
df = pd.concat([df1,df2],axis=0)
df

,邮件主题,邮件正文,分类
id,,,
1,非财务经理的财务管理-（沙盘模拟）,非财务纠淼牟莆窆芾 沙盘模拟 如何运用财务岳硖岣吖芾砑 课 程 背 景 每一位管理和技术人员...,垃圾
2,● 问一部魏宗万的电影名称,讲的是孔子后人的故事 一个老领导回到家乡 跟儿子感情不和 跟贪财的孙子孔为本和睦 老领导的弟...,正常
3,公司业务.代开发票！,尊敬的贵公司 财务 经理 负责人您好 我是深圳金海实业有限公司 广州 东莞 等省市有分公司 ...,垃圾
4,低点代开发票!,贵公司负责人 经理 财务 您好 深圳市华龙公司受多家公司委托向外低点代开部分增值税电脑发票 ...,垃圾
5,一边上网冲浪，一边赚钱，何乐而不为？,这是一封 格式信件 广告 网络电话包年卡 元 长途市话全包 最快的论坛邮址搜索专家 最好的邮...,垃圾
...,...,...,...
64606,优惠代开发票~!!!,贵公司负责人 经理 财务 您好 我公司是深圳市华源实业有限公司 本公司实力雄厚 全国各地均有...,垃圾
64607,代开发票验证后付款！,尊敬的商家朋友您好 我是深圳市裕华实业有限公司的 我司实力雄厚 有着良 好的社会关系 也有部...,垃圾
64608,优惠代开各类发票！,贵公司负责人 经理 财务 您好 我是深圳市康特实业有限公司 本公司在全国各地设有分公司 广州...,垃圾


#### 使用Python工具实现朴素贝叶斯
1. 导入数据并观察数据整体特点
    * 数据类别是否不平衡
    * 处理空值
    * 使用jieba等工具对原文分词
    * 删去无实际意义的词
2. 使用向量化工具进行向量化
3. 创建朴素贝叶斯算法对象
4. 调用fit方法进行拟合训练
5. 用向量化工具的transform方法对测试集文本向量化
6. 调用predict方法进行预测

In [10]:
df.groupby('分类').count() #count未计入正文为空的数据

,邮件主题,邮件正文
分类,,
垃圾,42035,40146
正常,19676,21692


In [11]:
# inplac表示直接在原数据集删除
df.dropna(subset=['邮件正文'], inplace=True)
df

,邮件主题,邮件正文,分类
id,,,
1,非财务经理的财务管理-（沙盘模拟）,非财务纠淼牟莆窆芾 沙盘模拟 如何运用财务岳硖岣吖芾砑 课 程 背 景 每一位管理和技术人员...,垃圾
2,● 问一部魏宗万的电影名称,讲的是孔子后人的故事 一个老领导回到家乡 跟儿子感情不和 跟贪财的孙子孔为本和睦 老领导的弟...,正常
3,公司业务.代开发票！,尊敬的贵公司 财务 经理 负责人您好 我是深圳金海实业有限公司 广州 东莞 等省市有分公司 ...,垃圾
4,低点代开发票!,贵公司负责人 经理 财务 您好 深圳市华龙公司受多家公司委托向外低点代开部分增值税电脑发票 ...,垃圾
5,一边上网冲浪，一边赚钱，何乐而不为？,这是一封 格式信件 广告 网络电话包年卡 元 长途市话全包 最快的论坛邮址搜索专家 最好的邮...,垃圾
...,...,...,...
64606,优惠代开发票~!!!,贵公司负责人 经理 财务 您好 我公司是深圳市华源实业有限公司 本公司实力雄厚 全国各地均有...,垃圾
64607,代开发票验证后付款！,尊敬的商家朋友您好 我是深圳市裕华实业有限公司的 我司实力雄厚 有着良 好的社会关系 也有部...,垃圾
64608,优惠代开各类发票！,贵公司负责人 经理 财务 您好 我是深圳市康特实业有限公司 本公司在全国各地设有分公司 广州...,垃圾


In [14]:
import jieba

# jieba.cut负责分词，为了后面方便使用sklearn向量化
#还需要将cut后的词语用空格连接起来（字符串的join方法）
df['分词'] = df['邮件正文'].map(lambda s: 
                          " ".join(jieba.cut(s)))
df

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/47/d594fr2n50z3wp56_35fy2n80000gn/T/jieba.cache
Loading model cost 1.078 seconds.
Prefix dict has been built successfully.


,邮件主题,邮件正文,分类,分词
id,,,,
1,非财务经理的财务管理-（沙盘模拟）,非财务纠淼牟莆窆芾 沙盘模拟 如何运用财务岳硖岣吖芾砑 课 程 背 景 每一位管理和技术人员...,垃圾,非 财务 纠淼 牟 莆 窆 芾 沙盘 模拟 如何 运用 财务 岳 硖 岣 吖 芾 砑...
2,● 问一部魏宗万的电影名称,讲的是孔子后人的故事 一个老领导回到家乡 跟儿子感情不和 跟贪财的孙子孔为本和睦 老领导的弟...,正常,讲 的 是 孔子 后人 的 故事 一个 老 领导 回到 家乡 跟 儿子 感情 不 和...
3,公司业务.代开发票！,尊敬的贵公司 财务 经理 负责人您好 我是深圳金海实业有限公司 广州 东莞 等省市有分公司 ...,垃圾,尊敬 的 贵 公司 财务 经理 负责人 您好 我 是 深圳 金海 实业 有限...
4,低点代开发票!,贵公司负责人 经理 财务 您好 深圳市华龙公司受多家公司委托向外低点代开部分增值税电脑发票 ...,垃圾,贵 公司 负责人 经理 财务 您好 深圳市 华龙 公司 受 多家 公司 委托...
5,一边上网冲浪，一边赚钱，何乐而不为？,这是一封 格式信件 广告 网络电话包年卡 元 长途市话全包 最快的论坛邮址搜索专家 最好的邮...,垃圾,这是 一封 格式 信件 广告 网络电话 包年 卡 元 长途 市话 全包 ...
...,...,...,...,...
64606,优惠代开发票~!!!,贵公司负责人 经理 财务 您好 我公司是深圳市华源实业有限公司 本公司实力雄厚 全国各地均有...,垃圾,贵 公司 负责人 经理 财务 您好 我 公司 是 深圳市 华源 实业 有限公...
64607,代开发票验证后付款！,尊敬的商家朋友您好 我是深圳市裕华实业有限公司的 我司实力雄厚 有着良 好的社会关系 也有部...,垃圾,尊敬 的 商家 朋友 您好 我 是 深圳市 裕华 实业 有限公司 的 我司 实力雄厚...
64608,优惠代开各类发票！,贵公司负责人 经理 财务 您好 我是深圳市康特实业有限公司 本公司在全国各地设有分公司 广州...,垃圾,贵 公司 负责人 经理 财务 您好 我 是 深圳市 康特 实业 有限公司 ...


In [15]:
#将百度停止词文件读入stopwords列表，每行读作一个字符串
stopwprds = []
with open('baidu-stopword.txt',encoding='utf-8') as f:
    stopwprds = f.readlines()

stopwprds = [s.strip() for s in stopwprds]
stopwprds

['--',
 '?',
 '“',
 '”',
 '》',
 '－－',
 'able',
 'about',
 'above',
 'according',
 'accordingly',
 'across',
 'actually',
 'after',
 'afterwards',
 'again',
 'against',
 "ain't",
 'all',
 'allow',
 'allows',
 'almost',
 'alone',
 'along',
 'already',
 'also',
 'although',
 'always',
 'am',
 'among',
 'amongst',
 'an',
 'and',
 'another',
 'any',
 'anybody',
 'anyhow',
 'anyone',
 'anything',
 'anyway',
 'anyways',
 'anywhere',
 'apart',
 'appear',
 'appreciate',
 'appropriate',
 'are',
 "aren't",
 'around',
 'as',
 "a's",
 'aside',
 'ask',
 'asking',
 'associated',
 'at',
 'available',
 'away',
 'awfully',
 'be',
 'became',
 'because',
 'become',
 'becomes',
 'becoming',
 'been',
 'before',
 'beforehand',
 'behind',
 'being',
 'believe',
 'below',
 'beside',
 'besides',
 'best',
 'better',
 'between',
 'beyond',
 'both',
 'brief',
 'but',
 'by',
 'came',
 'can',
 'cannot',
 'cant',
 "can't",
 'cause',
 'causes',
 'certain',
 'certainly',
 'changes',
 'clearly',
 "c'mon",
 'co',
 'com',


In [17]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df,random_state=42)

In [18]:
# 向量化
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer(stop_words=stopwprds)

# x_train的结构是：（行号，数字编码） 出现次数
x_train = vect.fit_transform(df_train['分词'])

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'aren', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'll', 'mon', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


执行fit或fit_transform之后，工具对象vect会给所有出现过的词语分别指定一个数字编码，该编码可以使用vocabulary_属性查看。

In [19]:
vect.vocabulary_

{'小心': 47668,
 '开车': 52999,
 '据统计': 64016,
 '眼中': 94260,
 '有泪': 72322,
 '出车祸': 21863,
 '执意': 60847,
 '晚餐': 70781,
 '一句': 403,
 '想见': 58095,
 '雀跃': 128058,
 '赴约': 116872,
 '女友': 42387,
 '迷恋': 120808,
 '不了': 3340,
 '恍然大悟': 57033,
 '相识': 93610,
 '两个': 5421,
 '日子': 69314,
 '怪不得': 56718,
 '这段': 120087,
 '不明不白': 3990,
 '感情': 58397,
 '挣脱': 63542,
 '原本': 28382,
 '不想': 3872,
 '说出': 114278,
 '那句话': 122430,
 '公司': 18598,
 '负责人': 115580,
 '经理': 101448,
 '财务': 115614,
 '您好': 57487,
 '深圳市': 83482,
 '海洋': 82843,
 '实业': 45612,
 '有限公司': 72514,
 '进项': 120355,
 '要冲': 111787,
 '增值税': 38349,
 '电脑': 91214,
 '发票': 29458,
 '海关': 82727,
 '代征': 12114,
 '专用': 4757,
 '缴款': 102442,
 '国税': 36461,
 '地税': 37310,
 '普通发票': 70886,
 '对外': 46835,
 '优惠': 13053,
 '代开': 12111,
 '合作': 31609,
 '另有': 30260,
 '最新': 71772,
 '运输': 119595,
 '数目': 67104,
 '大小': 40229,
 '衡量': 110767,
 '郑重': 122939,
 '承诺': 61265,
 '所用': 60046,
 '票据': 96099,
 '上网': 2945,
 '查询': 75523,
 '税务局': 97167,
 '抵扣': 62119,
 '验证': 131717,
 '信息': 15623,
 '长期有效': 1

In [21]:
from sklearn.naive_bayes import MultinomialNB

bys = MultinomialNB()

model = bys.fit(x_train, df_train['分类'])

In [22]:
df_test

,邮件主题,邮件正文,分类,分词
id,,,,
33638,Re: 放不下的感情- _ -,说不好 偶对大学里的爱情都不抱什么希望了 既然人家都有男朋友了 你也再找一个不就完了嘛 实在...,正常,说 不好 偶对 大学 里 的 爱情 都 不 抱 什么 希望 了 既然 人家 都 有 ...
29260,爱芙瑞健康美容加盟连锁现正火热加盟中,对爱芙瑞的宣传 朋友们最感兴趣的就是 总部保证加盟店开业十天内 营业额达到二万元 不足部分总...,垃圾,对 爱 芙瑞 的 宣传 朋友 们 最 感兴趣 的 就是 总部 保证 加盟店 开业 十...
6386,NaN,帮您将产品销售到全球各地 简单方便与外国人做生意 全球工商贸易网 是一个面向全世界各国家和地...,垃圾,帮 您 将 产品销售 到 全球 各地 简单 方便 与 外国人 做生意 全球 工商 贸...
40652,张晓景,我还是更喜欢大决战 月 日 周五 下午两点新闻与传播学院与八一电影制片厂将于大礼堂联合举办影...,正常,我 还是 更 喜欢 大决战 月 日 周五 下午 两点 新闻 与 传播学院 与...
55315,Re: 为什么大家盲目崇拜王垠？？？,照你这么说每天勤勤恳恳 任劳任怨的人就没有理想了 学生一定要按照自己的想法去做才是对的 而不...,正常,照 你 这么 说 每天 勤勤恳恳 任劳任怨 的 人 就 没有 理想 了 学生 一定 ...
...,...,...,...,...
52300,Re: 我倒觉得异地恋更需要耐心，需要克制,呵呵 我知道你的意思了 其实我来广州的时候 有女朋友 现在是异地恋 我心里现在主要还是合肥那...,正常,呵呵 我 知道 你 的 意思 了 其实 我来 广州 的 时候 有 女朋友 现...
41007,刘欢,北大学子主创的 综合交友日记网 敞开怀抱 用十万分的 真城欢幽 北大学子主创的 综合交友日记...,垃圾,北大 学子 主创 的 综合 交友 日记 网 敞开 怀抱 用 十万分 的 真城...
607,Re: 巨大的世界 微小一部分,年之痒 今天看到了 我大学的一个同学 毕业三年 微微有点发福 神情黯然的从我身边插身而过 我...,正常,年 之 痒 今天 看到 了 我 大学 的 一个 同学 毕业 三年 微微 有点...


In [23]:
x_test = vect.transform(df_test['分词'])


y_test = model.predict(x_test)

y_test

array(['正常', '垃圾', '垃圾', ..., '正常', '垃圾', '垃圾'], dtype='<U2')

In [24]:
from sklearn import metrics

print("Accuracy: ", metrics.accuracy_score(y_test, df_test['分类']))

Accuracy:  0.974385510996119
